In [22]:
import pandas as pd
import json
from bs4 import BeautifulSoup
from datetime import datetime

# Path to your JSON file
file_path = "/Users/ohamugochukwu/Documents/Oham-Job-Board/jobs.json"

# Method 1: Using json.load() to read the file and then convert to a DataFrame
with open(file_path, 'r') as file:
    data = json.load(file)

# Convert the JSON object to a DataFrame
df = pd.DataFrame(data['jobs'])
df.head()


,id,url,title,company_name,company_logo,category,tags,job_type,publication_date,candidate_required_location,salary,description,company_logo_url
0,1937616,https://remotive.com/remote-jobs/software-dev/...,"Senior Software Engineer, Identity & Access",Kunai,https://remotive.com/job/1937616/logo,Software Development,"[C++, golang, java, python]",full_time,2024-09-20T02:50:45,USA,,"<div class='""content-intro""'><p>Kunai is a fas...",NaN
1,1937543,https://remotive.com/remote-jobs/software-dev/...,Machine Learning Engineer,Red Cell Partners,https://remotive.com/job/1937543/logo,Software Development,"[data science, python]",full_time,2024-09-20T02:50:45,USA,"$165,000-$215,000","<div class='""content-intro""'><p>About Us</p>\n...",NaN
2,1937672,https://remotive.com/remote-jobs/all-others/rn...,RN Healthcare Advocate,Solace,https://remotive.com/job/1937672/logo,All others,"[video, documentation, healthcare, startup, ma...",contract,2024-09-20T00:50:54,USA,,"<p style=""min-height:1.5em""><em>Solace is a he...",NaN
3,1937596,https://remotive.com/remote-jobs/writing/conte...,Content Writer - Intern,Paytm,https://remotive.com/job/1937596/logo,Writing,[research],internship,2024-09-20T00:50:50,India,"10,000 inr per month",<div><strong>About Us: </strong></div>\n<div>P...,NaN
4,1937565,https://remotive.com/remote-jobs/finance-legal...,"Supervisor, Account receivable",Natera,https://remotive.com/job/1937565/logo,Finance / Legal,"[accounting, cloud, excel, oracle, security, j...",full_time,2024-09-20T00:50:41,USA,"$71,300 - $117,600 usd",<p><b>POSITION SUMMARY:</b> <br>The Accounts ...,NaN


In [27]:
import numpy as np

np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [26]:
from sqlalchemy import func
from database import get_db, Job



db = next(get_db())
latestDate = db.query(func.max(Job.PublicationDate)).scalar()

# Convert 'publication_date' to just the date part
df['publication_date'] = pd.to_datetime(df['publication_date']).dt.date

# Filter the DataFrame for rows where 'publication_date' <= '2024-09-18'
filtered_df = df[df['publication_date'] == latestDate]

# Show the filtered DataFrame
filtered_df.head()

2024-09-20


,id,url,title,company_name,company_logo,category,tags,job_type,publication_date,candidate_required_location,salary,description,company_logo_url
0,1937616,https://remotive.com/remote-jobs/software-dev/...,"Senior Software Engineer, Identity & Access",Kunai,https://remotive.com/job/1937616/logo,Software Development,"[C++, golang, java, python]",full_time,2024-09-20,USA,,"<div class='""content-intro""'><p>Kunai is a fas...",NaN
1,1937543,https://remotive.com/remote-jobs/software-dev/...,Machine Learning Engineer,Red Cell Partners,https://remotive.com/job/1937543/logo,Software Development,"[data science, python]",full_time,2024-09-20,USA,"$165,000-$215,000","<div class='""content-intro""'><p>About Us</p>\n...",NaN
2,1937672,https://remotive.com/remote-jobs/all-others/rn...,RN Healthcare Advocate,Solace,https://remotive.com/job/1937672/logo,All others,"[video, documentation, healthcare, startup, ma...",contract,2024-09-20,USA,,"<p style=""min-height:1.5em""><em>Solace is a he...",NaN
3,1937596,https://remotive.com/remote-jobs/writing/conte...,Content Writer - Intern,Paytm,https://remotive.com/job/1937596/logo,Writing,[research],internship,2024-09-20,India,"10,000 inr per month",<div><strong>About Us: </strong></div>\n<div>P...,NaN
4,1937565,https://remotive.com/remote-jobs/finance-legal...,"Supervisor, Account receivable",Natera,https://remotive.com/job/1937565/logo,Finance / Legal,"[accounting, cloud, excel, oracle, security, j...",full_time,2024-09-20,USA,"$71,300 - $117,600 usd",<p><b>POSITION SUMMARY:</b> <br>The Accounts ...,NaN


In [14]:
from sqlalchemy import func
from database import get_db, Job



db = next(get_db())
latestDate = db.query(func.max(Job.PublicationDate)).scalar()
print(latestDate)

# # Convert the string to a date object using strptime
# date_object = datetime.strptime(date_string, '%Y-%m-%d').date()


2024-09-20


In [121]:
# Drop the 'age' column
df.drop(columns=["company_logo_url"], inplace=True)

import re

def normalize_salary(salary):
    # Step 1: Remove any extra text
    salary = salary.lower()
    salary = re.sub(r'(plus.*|competitive.*|equity.*|stock.*|bonus.*|dependent.*|options.*|salary.*|remuneration.*|pre-ipo.*|commission.*|packages.*|bonus.*)', '', salary)
    
    # Step 2: Normalize ranges by converting various types of separators to '-'
    salary = salary.replace('to', '-').replace('–', '-').replace('—', '-').replace('–', '-')
    
    # Step 3: Handle 'k' notation (e.g., '50k' -> '50000')
    salary = re.sub(r'(\d+)k', lambda x: str(int(x.group(1)) * 1000), salary)
    
    # Step 4: Convert monthly and hourly rates to annual (assuming 12 months per year for monthly and 2080 hours per year for hourly)
    if 'hour' in salary:
        match = re.search(r'(\d+)-(\d+) (\$|\£|\€)? per hour', salary)
        if match:
            low, high = int(match.group(1)), int(match.group(2))
            salary = f'{int(((low + high) / 2) * 2080)}'
    
    if 'month' in salary:
        match = re.search(r'(\d+)-(\d+) (\$|\£|\€)? per month', salary)
        if match:
            low, high = int(match.group(1)), int(match.group(2))
            salary = f'{int(((low + high) / 2) * 12)}'
    
    # Step 5: Handle different currencies
    # For simplicity, let's assume all values are in USD or can be compared directly
    # salary = re.sub(r'[€£]', '$', salary)  # Convert other currencies to USD symbol
    
    # Step 6: Remove unwanted text such as '/year', etc.
    salary = re.sub(r'(per.*|/.*)', '', salary)
    
    # Step 7: Extract and format numeric values
    match = re.search(r'(\d+(?:,\d{3})*)', salary)
    if match:
        salary = f'{match.group(1).replace(",", "")}'
    else:
        salary = '0'  # Default value if nothing matched

    return salary


# Clean each salary string
cleaned_salaries = [normalize_salary(s) for s in df['salary']]

# Output cleaned list
# print(len(cleaned_salaries))

df['new_salary'] = cleaned_salaries

# Drop the 'age' column
df.drop(columns=["salary"], inplace=True)

# Dictionary specifying the old names and new names
new_column_names = {
    'id': 'Id',
    'url': 'URL',
    'title': 'Title',
    'company_name': 'CompanyName',
    'company_logo': 'CompanyLogo',
    'category': 'Category',
    'tags': 'Tags',
    'job_type': 'JobType',
    'publication_date': 'PublicationDate',
    'candidate_required_location': 'Location',
    'description': 'Description',
    'new_salary': 'Salary'
}

# Rename columns
df.rename(columns=new_column_names, inplace=True)

# Remove duplicates based on 'job_title' column
df = df.drop_duplicates(subset='Id')


def clean_html(html_content):
    """
    Cleans up HTML content by removing non-breaking spaces, unnecessary inline styles,
    and ensuring proper tag formatting.

    Parameters:
    html_content (str): The HTML content to clean.

    Returns:
    str: The cleaned HTML content.
    """
    if not html_content:
        return ''
    
    soup = BeautifulSoup(html_content, 'html.parser')

    # Remove empty paragraphs, spaces, and other unwanted content
    for tag in soup.find_all():
        # Remove unwanted inline styles and attributes
        tag.attrs = {}
        
        # Remove empty tags (if they contain no meaningful text or children)
        if tag.name == 'p' and (not tag.text.strip() or tag.text == '\xa0'):
            tag.decompose()

    # Clean up multiple newlines or unnecessary tags
    cleaned_html = str(soup).replace('\n', '').replace('\xa0', ' ').strip()

    return cleaned_html

def decode_unicode_escape(input_str):
    # Define a function to convert \uXXXX to the actual character
    def unicode_escape_to_char(match):
        hex_value = match.group(1)
        return chr(int(hex_value, 16))
    
    # Replace all \uXXXX with the actual character
    decoded_str = re.sub(r'\\u([0-9A-Fa-f]{4})', unicode_escape_to_char, input_str)
    
    return decoded_str

df['Description'] = df['Description'].apply(clean_html).apply(decode_unicode_escape)

df.head()


,Id,URL,Title,CompanyName,CompanyLogo,Category,Tags,JobType,PublicationDate,Location,Description,Salary
0,1937616,https://remotive.com/remote-jobs/software-dev/...,"Senior Software Engineer, Identity & Access",Kunai,https://remotive.com/job/1937616/logo,Software Development,"[C++, golang, java, python]",full_time,2024-09-20T02:50:45,USA,<div><p>Kunai is a fast-growing digital consul...,0
1,1937543,https://remotive.com/remote-jobs/software-dev/...,Machine Learning Engineer,Red Cell Partners,https://remotive.com/job/1937543/logo,Software Development,"[data science, python]",full_time,2024-09-20T02:50:45,USA,<div><p>About Us</p><p>Red Cell Partners is an...,165000
2,1937672,https://remotive.com/remote-jobs/all-others/rn...,RN Healthcare Advocate,Solace,https://remotive.com/job/1937672/logo,All others,"[video, documentation, healthcare, startup, ma...",contract,2024-09-20T00:50:54,USA,<p><em>Solace is a healthcare advocacy marketp...,0
3,1937596,https://remotive.com/remote-jobs/writing/conte...,Content Writer - Intern,Paytm,https://remotive.com/job/1937596/logo,Writing,[research],internship,2024-09-20T00:50:50,India,<div><strong>About Us: </strong></div><div>Pay...,10000
4,1937565,https://remotive.com/remote-jobs/finance-legal...,"Supervisor, Account receivable",Natera,https://remotive.com/job/1937565/logo,Finance / Legal,"[accounting, cloud, excel, oracle, security, j...",full_time,2024-09-20T00:50:41,USA,<p><b>POSITION SUMMARY:</b> <br/>The Accounts...,71300


In [122]:
from database import engine
from sqlalchemy.dialects.postgresql import insert



def insert_on_conflict_nothing(table, conn, keys, data_iter):
    # "a" is the primary key in "conflict_table"
    data = [dict(zip(keys, row)) for row in data_iter]
    stmt = insert(table.table).values(data).on_conflict_do_nothing(index_elements=["Id"])
    result = conn.execute(stmt)
    return result.rowcount

# Write the DataFrame to a PostgreSQL table using the imported engine
try:
    df.to_sql('Jobs', engine, if_exists='append', index=False, method=insert_on_conflict_nothing)
except Exception as e:
    print("An error occurred:", e)